In [1]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials
using Mosek#, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

include("../src/MinimumActionPath.jl")
using MAP

include("../src/GeomMinActPath.jl")
using gMAM

INFO: Recompiling stale cache file /cluster/home/rdb13/.julia/lib/v0.6/Optim.ji for module Optim.


# Nonlinear Examples

We will examine some nonlinear examples on which the SOS decomposition method is applied.

In [2]:
# A 1D system with a quartic potential
@polyvar x;    x = [x];
F1(X::Vector) = [X[1] - X[1]^3 + 0.1];
f1 = F1(x);
@time U1 = NormalSoS.normdecomp(f1,x, MosekSolver(),1)
NormalSoS.checknorm(f1,U1,x)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x^4, x^3, x^2, x, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 61              
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization pro

status = :Stall
bnd = DynamicPolynomials.Monomial{true}[x^4]
getvalue(ϵ) = [0.249961]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 69              
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic op

 30.573555 seconds (11.39 M allocations: 601.338 MiB, 1.93% gc time)


2.5271169070811736e-5

In [3]:
@show(U1);

U1 = 0.24998873431572866x^4 - 0.4999979210237132x^2 - 0.0999986771350169x + 28657.82240539591


In [3]:
# A 2D quartic system from Zhou et al (2012)
@polyvar x[1:2]
F2(X::Vector) = [-1.0 + 9.0X[1] - 2.0X[1]^3 + 9.0X[2] - 2.0X[2]^3;
      1.0 - 11.0X[1] + 2.0X[1]^3 + 11.0X[2] - 2.0X[2]^3];
f2 = F2(x);
@time Ueg2 = NormalSoS.normdecomp(f2,x, MosekSolver(),1)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1x2^3, x2^4, x1^3, x1^2x2, x1x2^2, x2^3, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 594             
  Cones                  : 0               
  Scalar variables       : 18              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        :

x1^4 + x1^2x2^2 + x2^4
getvalue(ϵ) = [0.220945, 0.560384, 0.214877]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 621             
  Cones                  : 0               
  Scalar variables       : 17              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem

0.5000000043330644x1^4 + 0.5000000078114762x2^4 - 4.999999972977872x1^2 + 0.999999824171771x1x2 - 4.999999897900875x2^2 + 0.9999999600165587x1 + 428.8074466955952

## Maier-Stein model

We now look at the widely studied Maier-Stein model which may offer either a pure gradient system or one for which an orthogonal decomposition does not exist.

In [2]:
# The Maier-Stein model
γ = 1.0;    μ = 1.0;
@polyvar x[1:2]
F(X::Vector) = [X[1] - X[1]^3 - γ*X[1]X[2]^2;
                 -μ*(X[1]^2 + 1)X[2]];
f = F(x);
V = -0.5*x[1]^2 + 0.25*x[1]^4 + 0.5γ*x[2]^2 + 0.5γ*x[1]^2*x[2]^2;
@time U = NormalSoS.normdecomp(f,x, MosekSolver(),2)
# NormalSoS.checknorm(f3,U3,x)

fg = -differentiate(U,x)
Fg(X::Vector) = [Float64(subs(fg[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fg[2],x[1]=>X[1], x[2]=>X[2]))];
fc = f + differentiate(U,x)
Fc(X::Vector) = [Float64(subs(fc[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fc[2],x[1]=>X[1], x[2]=>X[2]))];
U

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1^3, x1^2x2, x1x2^2, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 544             
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  

status = :Stall
bnd = x1^4 + x1^2x2^2 + x2^2
getvalue(ϵ) = [0.249899, 0.499988, 0.499987]
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(543) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(544) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(545) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(546) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(548) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(549) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(550) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(551) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(552) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements a

MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(543) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(544) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(545) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(546) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(548) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(549) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(550) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(551) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(552) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(553) of matrix 'A'.
Warning number 705 is disabled.
Problem


 33.681096 seconds (12.03 M allocations: 635.925 MiB, 2.12% gc time)


0.2499997993665902x1^4 + 0.499999985042302x1^2x2^2 - 0.5000001662620626x1^2 + 0.4999998360565416x2^2 + 3320.7468232552387

In [3]:
# Obtain the minimum action path
x₀ = [-1.0 0.0];  # Start point (row vector)
xₑ = [-1.0 1.0];
Tspan = 10.0;  # Time span
N = 400;
φ₀ = MAP.makepath(x₀,xₑ,N);
for ii=1:N-2
    φ₀[ii,2] = 0.1*randn();
end

σ = 1.0;
G(X::Vector) = σ*[1.0; 1.0];

# Use Optim to optimise over path φ
resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
res = [x₀; Optim.minimizer(resObj); xₑ];

Optimisation for T=10.00 gives S=2.00
Optimisation is converged.


In [4]:
# Find the SOS-predicted minimum action path and compare ΔU with gMAM
dt = 0.02;    M=500;
φ = zeros(Float64,M,2);
φ[1,:] = xₑ;
for ii=2:M
    φ[ii,:] = φ[ii-1,:] + (-Fc(φ[ii-1,:])+Fg(φ[ii-1,:]))dt;
    # println((Fc(φ[ii,:])-Fg(φ[ii,:]))dt)
end

Sm = gMAM.action(res,F,G);
Ss = gMAM.action(flipdim(φ,1),F,G);
Su = Float64(subs(U,x[1]=>xₑ[1],x[2]=>xₑ[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
@show(Sm);    @show(Ss);    @show(Su);

Sm = 1.0001392485358809
Ss = 1.0001352308537799
Su = 0.9999998210992089


In [ ]:
mapPlt = plot(φ[:,1],φ[:,2], line=2, marker=0,label="SoS");
plot!(mapPlt, res[:,1],res[:,2], line=0, marker=2, label="MAP")

### Quasipotential comparison

We now plot a comparison bewteen the true and SOS computed quasipotential along a line.

In [18]:
γ = 1.0;    μ = 2.5;
@polyvar x[1:2]
F(X::Vector) = [X[1] - X[1]^3 - γ*X[1]X[2]^2;
                 -μ*(X[1]^2 + 1)X[2]];
f = F(x);
@time U = NormalSoS.normdecomp(f,x, MosekSolver(),2)

Tspan = 10.0;  # Time span
N = 400;
x₀ = [-1.0 0.0];

xVec = collect(-1.5:0.1:0.0)
Smx = zeros(xVec);
for (idx,xi) in enumerate(xVec)
    xₑ = [xi 0];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    for ii=1:N-2
        φ₀[ii,2] = 0.1*randn();
    end
    
    # Find the optimal path
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    
    # Compute the true action
    Smx[idx] = gMAM.action(φ,F,G);
    # Su[idx] = Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2])) - Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])); 
end

xVec = collect(-1.5:0.1:1.5)
Smy = zeros(xVec);
for (idx,xi) in enumerate(xVec)
    xₑ = [-1.0 xi];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    for ii=1:N-2
        φ₀[ii,2] = 0.1*randn();
    end
    
    # Find the optimal path
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    
    # Compute the true action
    Smy[idx] = gMAM.action(φ,F,G);
    # Su[idx] = Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2])) - Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])); 
end

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1^3, x1^2x2, x1x2^2, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 544             
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  

status = :Stall
bnd = x1^4 + x1^2x2^2 + x2^2
getvalue(ϵ) = [0.249202, 0.511518, 1.24943]
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(543) of matrix 'A'.
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(544) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(545) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(546) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(548) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(549) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(550) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(551) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(552) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements ar


Optimisation is converged.
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.46
Optimisation not converged
Optimisation for T=10.00 gives S=0.24
Optimisation is converged.
Optimisation for T=10.

In [20]:
plt = plot(plot(ylim=(0.0,0.4)), plot(ylim=(0.0,2.5)), layout = @layout([a; b]), size=(400,800))

xVec = collect(-1.5:0.1:0.0)
Us = xi->Float64(subs(U,x[1]=>xi,x[2]=>x₀[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot!(plt[1], collect(xVec[1]:0.01:xVec[end]),Us, label="SOS")
plot!(plt[1], xVec,Smx, line=0, marker=3, label="MAP")

xVec = collect(-1.5:0.1:1.5)
Us = xi->Float64(subs(U,x[1]=>x₀[1],x[2]=>xi)) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot!(plt[2], collect(xVec[1]:0.01:xVec[end]),Us, label="SOS")
plot!(plt[2], xVec,Smy, line=0, marker=3, label="MAP")
Plots.svg("MaierStein3")

## Zhou's quadratic system

In [3]:
## Example 1: Quadratic system from Zhou et al (2012)
@polyvar x[1:2]
F(X::Vector) = [-X[1] + 2.0X[2]^2;
                -1.5X[1]*X[2] - 2.0X[2]];
f = F(x);
@time U = NormalSoS.normdecomp(f,x, MosekSolver(),2);

fg = -differentiate(U,x)
Fg(X::Vector) = [Float64(subs(fg[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fg[2],x[1]=>X[1], x[2]=>X[2]))];
fc = f + differentiate(U,x)
Fc(X::Vector) = [Float64(subs(fc[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fc[2],x[1]=>X[1], x[2]=>X[2]))];

plt = NormalSoS.plotlandscape(f,U,x,([-3 3],[-3 3]),true);    plot(plt)
NormalSoS.checknorm(f,U,x)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 101             
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 1
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective s

 22.478684 seconds (11.84 M allocations: 613.509 MiB, 1.89% gc time)


0.2735042736156235

In [4]:
# Obtain the minimum action path
x₀ = [0.0 0.0];  # Start point (row vector)
xₑ = [-1.0 0.8];
Tspan = 50.0;  # Time span
N = 400;
φ₀ = MAP.makepath(x₀,xₑ,N);
for ii=1:N-2
    φ₀[ii,2] = 0.1*randn();
end

σ = 1.0;
G(X::Vector) = σ*[1.0; 1.0];

# Use Optim to optimise over path φ
resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
res = [x₀; Optim.minimizer(resObj); xₑ];

Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation not converged


In [5]:
# Find the SOS-predicted minimum action path and compare ΔU with gMAM
dt = 0.02;    M=500;
φ = zeros(Float64,M,2);
φ[1,:] = xₑ;
for ii=2:M
    φ[ii,:] = φ[ii-1,:] + (-Fc(φ[ii-1,:])+Fg(φ[ii-1,:]))dt;
    # println((Fc(φ[ii,:])-Fg(φ[ii,:]))dt)
end

Sm = gMAM.action(res,F,G);
Ss = gMAM.action(flipdim(φ,1),F,G);
Su = Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])) - Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2]));
@show(Sm);    @show(Ss);    @show(Su);

Sm = 1.0488605186535092
Ss = 1.0493808409240384
Su = 0.9266666564578752


In [6]:
mapPlt = plot(φ[:,1],φ[:,2], line=2, marker=0,label="SoS");
plot!(mapPlt, res[:,1],res[:,2], line=2, marker=0, label="MAP")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.00 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
<polyline clip-path="url(#clip1402)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 53.1849,22.1145 50.3198,36.9974 48.6225,51.2905 47.9794,64.9932 48.2847,78.1097 49.4402,90.6487 51.3556,102.622 53.9478,114.045 57.1406,124.932 60.8646,135.304 
 65.0563,145.178 69.6584,154.575 74.6184,163.514 79.8893,172.016 85.4281,180.1 91.1964,187.786 97.1594,195.094 103.286,202.041 109.547,208.647 115.918,214.928 
 122.376,220.901 128.901,226.583 135.475,231.989 142.08,237.133 148.703,242.029 155.331,246.69 161.951,251.129 168.555,255.358 175.131,259.388 181.673,263.23 
 188.173,266.894 194.625,270.388 201.022,273.723 207.362,276.907 213.638,279.947 219.847,282.852 225.987,285.628 232.054,288.282 238.046,290.82 243.961,293.249 
 249.797,295.574 255.553,297.8 261.229,299.932 266.822,301.976 272.333,303.934 277.76,305.813 283.105,307.616 288.366,309.346 293.544,311.006 298.639,312.602 
 303.651,314.134 308.58,315.608 313.428,317.024 318.194,318.387 322.879,319.698 327.484,320.961 332.01,322.176 336.457,323.347 340.826,324.475 345.118,325.563 
 349.334,326.611 353.475,327.623 357.542,328.599 361.536,329.541 365.457,330.451 369.307,331.329 373.087,332.178 376.798,332.999 380.44,333.792 384.016,334.559 
 387.525,335.301 390.969,336.02 394.349,336.715 397.666,337.388 400.92,338.04 404.114,338.671 407.248,339.283 410.322,339.876 413.339,340.452 416.298,341.009 
 419.201,341.551 422.049,342.076 424.843,342.586 427.584,343.081 430.272,343.561 432.909,344.028 435.496,344.482 438.033,344.923 440.521,345.352 442.961,345.769 
 445.355,346.174 447.702,346.568 450.004,346.952 452.262,347.326 454.476,347.689 456.647,348.043 458.776,348.388 460.864,348.724 462.912,349.051 464.919,349.37 
 466.888,349.681 468.819,349.984 470.711,350.279 472.567,350.567 474.387,350.848 476.172,351.123 477.921,351.39 479.637,351.651 481.319,351.906 482.968,352.155 
 484.585,352.399 486.17,352.636 487.724,352.868 489.248,353.095 490.742,353.316 492.207,353.533 493.643,353.744 495.051,353.951 496.431,354.154 497.784,354.352 
 499.11,354.545 500.411,354.735 501.686,354.92 502.936,355.101 504.161,355.279 505.362,355.452 506.54,355.623 507.694,355.789 508.826,355.952 509.936,356.112 
 511.023,356.268 512.089,356.422 513.134,356.572 514.159,356.719 515.163,356.863 516.148,357.005 517.113,357.143 518.059,357.279 518.986,357.412 519.895,357.543 
 520.787,357.671 521.66,357.797 522.516,357.92 523.356,358.041 524.179,358.159 524.985,358.276 525.776,358.39 526.551,358.502 527.31,358.612 528.055,358.72 
 528.785,358.826 529.5,358.93 530.202,359.032 530.889,359.133 531.563,359.231 532.223,359.328 532.871,359.423 533.505,359.517 534.127,359.608 534.737,359.698 
 535.335,359.787 535.921,359.874 536.495,359.96 537.058,360.044 537.609,360.126 538.15,360.207 538.68,360.287 539.199,360.366 539.709,360.443 540.208,360.519 
 540.697,360.593 541.177,360.667 541.647,360.739 542.107,360.81 542.559,360.879 543.001,360.948 543.435,361.015 543.86,361.082 544.277,361.147 544.686,361.211 
 545.086,361.274 545.478,361.337 545.863,361.398 546.24,361.458 546.61,361.517 546.972,361.575 547.327,361.633 547.675,361.689 548.016,361.745 548.35,361.8 
 548.678,361.853 548.999,361.906 549.314,361.959 549.622,362.01 549.924,362.061 550.221,362.11 550.511,362.159 550.796,362.208 551.075,362.255 551.348,362.302 
 551.616,362.348 551.879,362.393 552.137,362.438 552.389,362.482 552.636,362.525 552.879,362.568 553.116,362.61 553.349,362.652 553.578,362.692 553.801,362.733 
 554.021,362.772 554.236,362.811 554.446,362.85 554.653,362.888 554.855,362.925 555.053,362.962 555.248,362.998 555.438,363.033 555.625,363.069 555.808,363.103 
 555.987,363.137 556.163,363.171 556.335,363.204 556.504,363.237 556.67,363.269 556.832,363.301 556.991,363.332 557.147,363.363 557.3,363

### Quasipotential comparison

We now plot a comparison bewteen the true and SOS computed quasipotential along a line.

In [17]:
Tspan = 50.0;  # Time span
N = 400;
xVec = collect(0.0:0.5:4.0);
Sm = zeros(xVec);

x₀ = [0.0 0.0];
for (idx,xi) in enumerate(xVec)
    xₑ = [xi 0.0];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    
    # Find the optimal path
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    
    # Compute the true action
    Sm[idx] = gMAM.action(φ,F,G);
    # Su[idx] = Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2])) - Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])); 
end
Sm[1] = 0.0;

Us = xi->Float64(subs(U,x[1]=>xi,x[2]=>x₀[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot(collect(xVec[1]:0.01:xVec[end]),Us, label="MAP")
plot!(xVec,Sm, line=0, marker=2, label="SoS")

Optimisation for T=50.00 gives S=0.00
Optimisation is converged.
Optimisation for T=50.00 gives S=0.25
Optimisation is converged.
Optimisation for T=50.00 gives S=1.00
Optimisation is converged.
Optimisation for T=50.00 gives S=2.25
Optimisation is converged.
Optimisation for T=50.00 gives S=4.00
Optimisation is converged.
Optimisation for T=50.00 gives S=6.25
Optimisation is converged.
Optimisation for T=50.00 gives S=9.00
Optimisation is converged.
Optimisation for T=50.00 gives S=12.25
Optimisation is converged.
Optimisation for T=50.00 gives S=16.00
Optimisation is converged.


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
<polyline clip-path="url(#clip2402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.2276,365.565 39.5433,365.563 40.8591,365.557 42.1748,365.546 43.4906,365.531 44.8063,365.511 46.122,365.488 47.4378,365.46 48.7535,365.428 50.0693,365.391 
 51.385,365.351 52.7008,365.305 54.0165,365.256 55.3323,365.202 56.648,365.144 57.9638,365.082 59.2795,365.016 60.5953,364.945 61.911,364.87 63.2267,364.79 
 64.5425,364.707 65.8582,364.619 67.174,364.526 68.4897,364.43 69.8055,364.329 71.1212,364.224 72.437,364.114 73.7527,364 75.0685,363.882 76.3842,363.76 
 77.7,363.633 79.0157,363.502 80.3314,363.367 81.6472,363.228 82.9629,363.084 84.2787,362.936 85.5944,362.783 86.9102,362.627 88.2259,362.466 89.5417,362.3 
 90.8574,362.131 92.1732,361.957 93.4889,361.779 94.8047,361.596 96.1204,361.409 97.4361,361.218 98.7519,361.023 100.068,360.823 101.383,360.619 102.699,360.411 
 104.015,360.199 105.331,359.982 106.646,359.761 107.962,359.535 109.278,359.306 110.594,359.072 111.909,358.834 113.225,358.591 114.541,358.344 115.857,358.093 
 117.172,357.838 118.488,357.578 119.804,357.314 121.12,357.045 122.435,356.773 123.751,356.496 125.067,356.215 126.383,355.929 127.698,355.639 129.014,355.345 
 130.33,355.047 131.646,354.744 132.961,354.437 134.277,354.126 135.593,353.811 136.909,353.491 138.224,353.167 139.54,352.838 140.856,352.505 142.172,352.168 
 143.487,351.827 144.803,351.482 146.119,351.132 147.434,350.777 148.75,350.419 150.066,350.056 151.382,349.689 152.697,349.318 154.013,348.942 155.329,348.562 
 156.645,348.178 157.96,347.789 159.276,347.397 160.592,347 161.908,346.598 163.223,346.192 164.539,345.782 165.855,345.368 167.171,344.95 168.486,344.527 
 169.802,344.099 171.118,343.668 172.434,343.232 173.749,342.792 175.065,342.348 176.381,341.899 177.697,341.446 179.012,340.989 180.328,340.528 181.644,340.062 
 182.96,339.592 184.275,339.117 185.591,338.639 186.907,338.156 188.223,337.668 189.538,337.177 190.854,336.681 192.17,336.181 193.486,335.676 194.801,335.168 
 196.117,334.655 197.433,334.137 198.749,333.616 200.064,333.09 201.38,332.56 202.696,332.025 204.012,331.486 205.327,330.943 206.643,330.396 207.959,329.844 
 209.275,329.288 210.59,328.728 211.906,328.163 213.222,327.595 214.538,327.021 215.853,326.444 217.169,325.862 218.485,325.276 219.801,324.686 221.116,324.091 
 222.432,323.492 223.748,322.889 225.064,322.282 226.379,321.67 227.695,321.054 229.011,320.434 230.326,319.809 231.642,319.18 232.958,318.547 234.274,317.909 
 235.589,317.267 236.905,316.621 238.221,315.971 239.537,315.316 240.852,314.657 242.168,313.994 243.484,313.326 244.8,312.654 246.115,311.978 247.431,311.298 
 248.747,310.613 250.063,309.924 251.378,309.231 252.694,308.533 254.01,307.831 255.326,307.125 256.641,306.414 257.957,305.7 259.273,304.98 260.589,304.257 
 261.904,303.529 263.22,302.797 264.536,302.061 265.852,301.321 267.167,300.576 268.483,299.827 269.799,299.073 271.115,298.315 272.43,297.553 273.746,296.787 
 275.062,296.016 276.378,295.242 277.693,294.462 279.009,293.679 280.325,292.891 281.641,292.099 282.956,291.303 284.272,290.502 285.588,289.697 286.904,288.888 
 288.219,288.074 289.535,287.256 290.851,286.434 292.167,285.608 293.482,284.777 294.798,283.942 296.114,283.103 297.43,282.259 298.745,281.411 300.061,280.559 
 301.377,279.703 302.693,278.842 304.008,277.977 305.324,277.107 306.64,276.234 307.956,275.356 309.271,274.473 310.587,273.587 311.903,272.696 313.219,271.801 
 314.534,270.902 315.85,269.998 317.166,269.09 318.481,268.178 319.797,267.261 321.113,266.34 322.429,265.415 323.744,264.485 325.06,263.552 326.376,262.614 
 327.692,261.671 329.007,260.725 330.323,259.774 331.639,258.819 332.955,257.859 334.27,256.895 335.586,255.927 336.902,254.955 338.218,253.978 339.533,252.997 
 340.849,252.0

In [39]:
xVec = -collect(0.0:0.24:1.6)

7-element Array{Float64,1}:
 -0.0 
 -0.24
 -0.48
 -0.72
 -0.96
 -1.2 
 -1.44